# Model serialization

```{autolink-concat}

```

This page demonstrates a strategy for exporting an amplitude model with its suggested parameter defaults to disk and loading it back into memory later on for computations with the computational backend.


In [ ]:
import logging
import os
import pickle
from textwrap import shorten

import cloudpickle
import jax.numpy as jnp
import sympy as sp
from ampform.sympy import cached
from IPython.display import Markdown
from tensorwaves.function.sympy import create_function

from polarimetry.io import mute_jax_warnings
from polarimetry.lhcb import load_model
from polarimetry.lhcb.particle import load_particles

mute_jax_warnings()

NO_LOG = "EXECUTE_NB" in os.environ
if NO_LOG:
    logging.disable(logging.CRITICAL)

## Export model


In [ ]:
data_dir = "../../data"
particles = load_particles(f"{data_dir}/particle-definitions.yaml")
model = load_model(f"{data_dir}/model-definitions.yaml", particles, model_id=0)
unfolded_intensity_expr = cached.unfold(model)

In [ ]:
free_parameters = {
    symbol: value
    for symbol, value in model.parameter_defaults.items()
    if isinstance(symbol, sp.Indexed)
    if "production" in str(symbol)
}
fixed_parameters = {
    symbol: value
    for symbol, value in model.parameter_defaults.items()
    if symbol not in free_parameters
}
subs_intensity_expr = cached.xreplace(unfolded_intensity_expr, fixed_parameters)

In [ ]:
def sigma3via12() -> dict[sp.Symbol, sp.Expr]:
    s1, s2, s3 = sp.symbols("sigma1:4", nonnegative=True)
    m0, m1, m2, m3 = sp.symbols("m:4", nonnegative=True)
    return {s3: m1**2 + m2**2 + m3**2 + m0**2 - s1 - s2}

In [ ]:
dict_forms = {
    "intensity_expr": unfolded_intensity_expr,
    "variables": {k: v.doit() for (k, v) in model.variables.items()},
    "parameter_defaults": model.parameter_defaults,
    "sigma3": sigma3via12(),
}

In [ ]:
filename = "exported-model.pkl"
with open(filename, "wb") as f:
    cloudpickle.dump(dict_forms, f)

## Import model

The model is saved in a Python {obj}`dict` and to a {mod}`pickle` file. The dictionary contains a SymPy expressions for the model and suggested parameter default values. These parameter and variable symbols are substituted using the `fully_substitute()` function.


In [ ]:
def load_model(filename: str) -> dict:
    if not os.path.exists(filename):
        msg = f"The input file not found at ${filename}"
        raise ValueError(msg)
    with open(filename, "rb") as f:
        return pickle.load(f)

In [ ]:
def fully_substitute(model_description: dict) -> sp.Expr:
    return (
        model_description["intensity_expr"]
        .xreplace(model_description["variables"])
        .xreplace(model_description["sigma3"])
        .xreplace(model_description["parameter_defaults"])
    )

In [ ]:
imported_model = load_model(filename)
intensity_on_2vars = fully_substitute(imported_model)

### Compilation

The resulting symbolic expression depends on two variables:

- $\sigma_1 = m_{K\pi}^2$, mass of the $K^- \pi^+$ system, and
- $\sigma_2 = m_{pK}^2$, mass of the $p K^-$ system.

This expression is turned into a numerical function by either {func}`~sympy.utilities.lambdify.lambdify`, using {doc}`JAX<jax:index>` as a computational backend.

For {mod}`sympy` backend the position argument are used.


In [ ]:
s12 = sp.symbols("sigma1:3", nonnegative=True)
assert intensity_on_2vars.free_symbols == set(s12)

func = sp.lambdify(s12, intensity_on_2vars)

In [ ]:
func(1.0, 3.0), func(1.1, 3.2)

The compilation to JAX is facilitated by {mod}`tensorwaves`:


In [ ]:
density = create_function(intensity_on_2vars, backend="jax")

In [ ]:
density({"sigma1": jnp.array([1.0, 1.1]), "sigma2": jnp.array([3.0, 3.2])})

## Serialization with `srepr`


SymPy expressions can directly be serialized to Python code as well, with the function [`srepr()`](https://docs.sympy.org/latest/modules/printing.html#sympy.printing.repr.srepr). For the full intensity expression, we can do so with:


In [ ]:
%%time
eval_str = sp.srepr(unfolded_intensity_expr)

In [ ]:
n_nodes = sp.count_ops(unfolded_intensity_expr)
byt = len(eval_str.encode("utf-8"))
mb = f"{1e-6 * byt:.2f}"
rendering = shorten(eval_str, placeholder=" ...", width=85)
src = f"""
This serializes the intensity expression of {n_nodes:,d} nodes
to a string of **{mb} MB**.

```
{rendering}
```
"""
Markdown(src)

It is up to the user, however, to import the classes of each exported node before the string can be unparsed with [`eval()`](https://docs.python.org/3/library/functions.html#eval) (see [this comment](https://github.com/ComPWA/polarimetry/issues/20#issuecomment-1809840854)).


In [ ]:
eval(eval_str)

In the case of this intensity expression, it is sufficient to import all definition from the main `sympy` module and the `Str` class.


In [ ]:
from sympy import *  # noqa: F403
from sympy.core.symbol import Str  # noqa: F401

In [ ]:
%%time
eval_imported_intensity_expr = eval(eval_str)

Notice how the imported expression is **exactly the same** as the serialized one, including assumptions:


In [ ]:
assert eval_imported_intensity_expr == unfolded_intensity_expr
assert hash(eval_imported_intensity_expr) == hash(unfolded_intensity_expr)

Optionally, the `import` statements can be embedded into the string. The parsing is then done with [`exec()`](https://docs.python.org/3/library/functions.html#exec) instead:


In [ ]:
exec_str = f"""\
from sympy import *
from sympy.core.symbol import Str

def get_intensity_function() -> Expr:
    return {eval_str}
"""

In [ ]:
exec_filename = "exported_intensity_model.py"
with open(exec_filename, "w") as f:
    f.write(exec_str)

In [ ]:
src = f"""
See {{download}}`{exec_filename}` for the exported model.
"""
Markdown(src)

In [ ]:
%%time
exec(exec_str)
exec_imported_intensity_expr = get_intensity_function()  # noqa: F405

In [ ]:
assert exec_imported_intensity_expr == unfolded_intensity_expr
assert hash(exec_imported_intensity_expr) == hash(unfolded_intensity_expr)

:::{note}
The load time is faster due to caching within SymPy.
:::


## Python package


[![PyPI package](https://badge.fury.io/py/polarimetry-lc2pkpi.svg)](https://pypi.org/project/polarimetry-lc2pkpi)
[![Supported Python versions](https://img.shields.io/pypi/pyversions/polarimetry-lc2pkpi)](https://pypi.org/project/polarimetry-lc2pkpi)

As noted on the [main page](../index.ipynb), the source code for this analysis is available as a Python package on [PyPI](https://pypi.org) and can be installed as follows.

```bash
pip install polarimetry-lc2pkpi
```

:::{autolink-concat}
:::

Each of the models can then simply be imported as


In [ ]:
from ampform_dpd.io import simplify_latex_rendering

simplify_latex_rendering()

In [ ]:
import polarimetry

model = polarimetry.published_model("Default amplitude model")
model.intensity.cleanup()

The expressions have to be converted to a numerical function to evaluate them over larger data samples. There are several ways of doing this (such as [algebraically substituting the parameter values first](https://github.com/ComPWA/polarimetry/issues/336#issuecomment-1905795986)), but it depends on your application what is best. Here's a small example where we want to evaluate the model over a set of data points on the Dalitz plane. We first 'unfold' the main intensity expression and lambdify it to a numerical function with [JAX](https://jax.rtfd.io) as computational backend.


In [ ]:
from ampform_dpd.io import cached

intensity_expr = cached.unfold(model)
intensity_func = cached.lambdify(intensity_expr, parameters=model.parameter_defaults)

Now, let's say we have [some data sample](https://github.com/ComPWA/polarimetry/issues/336#issuecomment-1904067511) containing generated phase space data points in the Dalitz plane.


In [ ]:
import urllib.request
from io import BytesIO
from zipfile import ZipFile

url = "https://github.com/ComPWA/polarimetry/files/14123343/lc2pkpi_phsp.zip"
with urllib.request.urlopen(url) as response:
    zipfile = ZipFile(BytesIO(response.read()))
zipfile.extractall()

In [ ]:
import uproot

with uproot.open("lc2pkpi_phsp.root") as root_file:
    events = root_file["Lc2ppiK"]
    df = events.arrays(library="pd")
df

Here, we have data points for the two Mandelstam variables $\sigma_1$ and $\sigma_2$. The {attr}`~ampform_dpd.AmplitudeModel.invariants` attribute of the amplitude model provides symbolic expressions for how to compute the third Mandelstam. In combination with the {attr}`~ampform_dpd.AmplitudeModel.variables` and {attr}`~ampform_dpd.AmplitudeModel.parameter_defaults` attributes, we can create a data transformer for computing helicity angles and DPD alignment angles.


In [ ]:
from ampform.io import aslatex
from IPython.display import Math

Math(aslatex(model.invariants))

In [ ]:
import sympy as sp
from tensorwaves.data.transform import SympyDataTransformer

*_, (s3, s3_expr) = model.invariants.items()
definitions = model.variables
definitions[s3] = s3_expr
definitions = {
    symbol: expr.xreplace(definitions).xreplace(model.parameter_defaults)
    for symbol, expr in definitions.items()
}
data_transformer = SympyDataTransformer.from_sympy(definitions, backend="jax")

Finally, we can create an input {obj}`~tensorwaves.interface.DataSample` that we can feed to the numerical function for the amplitude model.


In [ ]:
data = {
    "sigma1": df["msq_piK"].to_numpy(),
    "sigma2": df["msq_Kp"].to_numpy(),
}
completed_data = data_transformer(data)
completed_data.update(data)
intensity_func(completed_data)